In [ ]:
# Import required modules
import pandas as pd
import re                                           # regular expressions
import nltk
import nltk.corpus as corpus                        # nltk package for reading files
nltk.download('stopwords')
from nltk.corpus import stopwords as stopwords      # words such as 'the','be','an', that don't significantly impact the sentiment
from nltk.stem import WordNetLemmatizer             # Groups words
from nltk.tokenize import word_tokenize             # Splits text into tokens
import collections

# Will be used later on
import matplotlib as mpl
import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()
# import sklearn
# import numpy as np

In [89]:
# Filter out all unlabelled sentiment
p_df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/tweets_labelled.csv', sep=';',names=['created_at','text','sentiment']).query('sentiment == "positive"')
n_df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/tweets_labelled.csv', sep=';',names=['created_at','text','sentiment']).query('sentiment == "negative"')

# Debug Print
# display(p_df)
# display(n_df)

# Combine dataframes
frames = [p_df, n_df]
df = pd.concat(frames)

# Sort by date
# https://www.geeksforgeeks.org/how-to-sort-a-pandas-dataframe-by-date/
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.sort_values(by="created_at",ascending=True)
# Debug Print
display(df)


,created_at,text,sentiment
14135,2020-04-09 00:00:03+00:00,"$HEXO target price is $2.21, HEXO has a upside...",positive
13047,2020-04-09 02:23:18+00:00,#germany issuing €130.5b in 7 &amp; 15-yr #bon...,positive
12476,2020-04-09 03:59:33+00:00,Wells Fargo &amp; Co Reiterates “Buy” Rating f...,positive
11955,2020-04-09 05:58:02+00:00,This is not a favorable time to trade big. Why...,negative
6998,2020-04-09 14:42:56+00:00,"RT @amir: New: Microsoft’s rival to AWS, calle...",positive
...,...,...,...
929773,2020-07-16 16:11:19+00:00,Picked $amzn 3300 for earnings around 22. Shou...,positive
929272,2020-07-16 16:44:54+00:00,$AMZN $2992 you’re welcome,positive
928299,2020-07-16 17:50:52+00:00,UnitedHealth Group $UNH Price Target Raised to...,positive
927482,2020-07-16 18:49:34+00:00,Goldman Sachs Group $GS Price Target Raised to...,positive


In [90]:
# Grab the top 25 stocks
ticker_pattern = re.compile(r'(^\$[A-Z]+|^\$ES_F)')

# Break the string into a dictionary
ticker_freq_dict = collections.defaultdict(int)
# ticker_dict = collections.defaultdict(str)

ticker = ""
entry = ""
for text in df['text']:
    for word in text.split():
        if ticker_pattern.fullmatch(word) is not None:
            ticker_freq_dict["$" + word[1:]] += 1
    #         ticker = "$"+ word[1:]
    #         if ticker not in ticker_dict:
    #             ticker_dict[ticker] = []
    #         entry = entry + word
    # ticker_dict[ticker].append(entry)  
    
# ticker_freq_df = pd.DataFrame.from_dict(
#     ticker_freq_dict, orient='index', columns=['freq'])\
#     .sort_values('freq', ascending=False).head(25)

# Only store the top 25
ticker_freq_df = pd.DataFrame.from_dict(
    ticker_freq_dict, orient='index', columns=['freq'])\
    .nlargest(25, 'freq')
display(ticker_freq_df)


,freq
$SPX,132
$AMZN,80
$AAPL,79
$SPY,69
$FB,59
$TSLA,40
$MSFT,32
$QQQ,29
$NFLX,27
$JPM,22


In [91]:
# Create a dictionary of strings to save each stock within the dictionary
ticker_dict = collections.defaultdict(str)

# Iterate over the string
for text in df['text']:
    for ticker in ticker_freq_df.index:
        if text.find(ticker) > 0:
            if ticker not in ticker_dict:
                ticker_dict[ticker] = [text]
                continue
            else:
                ticker_dict[ticker].append(text)
                continue

# Print out the dictionary to json file for debugging purposes
import json

with open('ticker_dict.json','w') as fout:
    json.dump(ticker_dict, fout, indent=4)

# Confirm the correct amount of tweets have been written to the file
for ticker in ticker_freq_df.index:
    if len(ticker_dict[ticker]) != ticker_freq_dict[ticker]:
        print("ERROR: " + ticker + " expected: " + str(ticker_freq_dict[ticker]) + ", Actual: " + str(len(ticker_dict[ticker])))

with open('entry.json', 'w') as fout:
    json.dump(ticker_dict['$JNJ'], fout, indent=4)

ERROR: $SPX expected: 132, Actual: 119
ERROR: $AMZN expected: 80, Actual: 77
ERROR: $AAPL expected: 79, Actual: 77
ERROR: $SPY expected: 69, Actual: 66
ERROR: $FB expected: 59, Actual: 62
ERROR: $TSLA expected: 40, Actual: 41
ERROR: $MSFT expected: 32, Actual: 33
ERROR: $QQQ expected: 29, Actual: 31
ERROR: $NFLX expected: 27, Actual: 28
ERROR: $JPM expected: 22, Actual: 20
ERROR: $ES_F expected: 20, Actual: 17
ERROR: $NVDA expected: 19, Actual: 18
ERROR: $BA expected: 17, Actual: 30
ERROR: $GOOGL expected: 16, Actual: 17
ERROR: $BTC expected: 16, Actual: 13
ERROR: $ZM expected: 15, Actual: 11
ERROR: $T expected: 15, Actual: 77
ERROR: $IWM expected: 15, Actual: 17
ERROR: $GOOG expected: 15, Actual: 31
ERROR: $BAC expected: 13, Actual: 12
ERROR: $BYND expected: 11, Actual: 9
ERROR: $INTC expected: 11, Actual: 10
ERROR: $JNJ expected: 10, Actual: 11


In [ ]:
# Return a weight from 0-1 based on the percentage of positive tweets
# ticker_dict = {
#     "$SPX": [
#         (timestamp, "positive"),
#         (timestamp, "negative"),
#         ...
#     ],

# }